In [1]:
# Mount data from drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# Install packages not included in the colab default environment
%%capture
%pip install geopandas
%pip install earthpy as et
!pip install "rioxarray==0.3.1"
!pip install "xarray==0.18.2"

In [3]:
import matplotlib.pyplot as plt
import geopandas as gpd
import fiona
import glob
import os
import earthpy.spatial as es
import earthpy.plot as ep
import rasterio as rio
import rioxarray as rxr
import xarray as xr

In [4]:
# Path to Geotiffs of chemical traits
geotiffs_path = "/content/drive/MyDrive/GeoTiff/"


In [5]:
# Prepare layers to be stacked using glob and sort

layers_data_path = "/content/drive/MyDrive/GeoTiff/*.tif"
stack_band_paths = glob.glob(layers_data_path)
stack_band_paths.sort()

# Create output directory and the output path

output_dir = "/content/drive/MyDrive/outputs_watersheds/"
if os.path.isdir(output_dir) == False:
    os.mkdir(output_dir)

raster_out_path = os.path.join(output_dir, "raster.tiff")

In [6]:
#Define function for opening with rioxarray 

def open_clean_bands(band_path,
                     crop_bound,
                     valid_range=None,
                     variable=None):
    # YOUR CODE HERE

    """Open and clean a single landsat band .

    Parameters
    -----------
    band_path:string A path to the array to be opened
    crop_bound:geopandas GeoDataFrame
    A geopandas dataframe to be used to crop the raster data using rioxarray clip().
    valid_range:tuple (optional)
     A tuple of min and max range of values for the data. Default = None

    Returns
    -----------
     band : xarray DataArray
        An xarray DataArray clipped to a crop boundary and masked if a range is given
    """

    crop_bound_box = [box(*crop_bound.bounds.loc[0])]

    try:
        band = rxr.open_rasterio(band_path,
                                 masked=True,
                                 variable=variable, 
                                 parse_coordinates=False).rio.clip(crop_bound_box,
                                                             all_touched=True,
                                                             from_disk=True).squeeze()
    except:
        raise ValueError(
            "Oops - I couldn't clip your data. This may be due to a crs error.")

    # Only mask the data to the valid range if a valid range tuple is provided
    if valid_range is not None:
        mask = ((band < valid_range[0]) | (band > valid_range[1]))
        band = band.where(~xr.where(mask, True, False))

    return band

#Define combine tif function
def combine_tifs(tif_list):
    """A function that combines a list of tifs in the same CRS
    and of the same extent into an xarray object

    Parameters
    ----------
    tif_list : list
        A list of paths to the tif files that you wish to combine.
        
    Returns
    -------
    An xarray object with all of the tif files in the listmerged into 
    a single object.

    """

    out_xr=[]
    for i, tif_path in enumerate(tif_list):
        out_xr.append(rxr.open_rasterio(tif_path, masked=True)[1].squeeze())
        
        out_xr[i]["band"]=i+1
    (out_xr,) = xr.broadcast(*out_xr)
    
    return xr.concat(out_xr, dim="band") 


In [7]:
# Define path to shape file of boundary crop
rr_watershed_boundary_path = os.path.join("GeoTiff",
                                          "RR_HU8",
                                          "RR_HU8_polygon.shp")

In [8]:
final_stack = open_clean_bands(stack_band_paths)


TypeError: ignored